In [1]:
using Knet
Knet.gpu(0)

0

In [2]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", readstring(`nvidia-smi --query-gpu=name --format=csv,noheader`))

OS: Linux
Julia: 0.6.0
Knet: 0.8.5+
GPU: NVS 310
TITAN X (Pascal)



In [3]:
# define model
function initmodel(; atype=KnetArray, dtype=Float32, winit=xavier, binit=zeros)
    w(dims...)=atype(winit(dtype,dims...))
    b(dims...)=atype(binit(dtype,dims...))
    return Any[
        w(3,3,3,50), b(1,1,50,1),
        w(3,3,50,50), b(1,1,50,1),
        w(3,3,50,100), b(1,1,100,1),
        w(3,3,100,100), b(1,1,100,1),
        w(512,6400), b(512,1),
        w(10,512), b(10,1)
    ]
end;

In [4]:
#define loss and its gradient
function predict(w,x)
    convbias(x,w,b) = conv4(w,x;padding=1) .+ b
    fc(x,w,b) = w * mat(x) .+ b;
    x = relu.(convbias(x,w[1],w[2]))
    x = relu.(pool(convbias(x,w[3],w[4])))
    x = dropout(x,0.25)
    x = relu.(convbias(x,w[5],w[6]))
    x = relu.(pool(convbias(x,w[7],w[8])))
    x = dropout(x,0.25)
    x = relu.(fc(x,w[9],w[10]))
    x = dropout(x,0.5)
    return fc(x,w[11],w[12])
end

loss(w,x,y)=nll(predict(w,x),y) # nll: negative log likelihood
lossgradient = grad(loss);

In [8]:
xtrn = randn(Array{Float32}, 32,32,3,50000);
ytrn = ones(Array{UInt8}, 50000);

LoadError: [91mMethodError: no method matching randn(::MersenneTwister, ::Type{Array{Float32,N} where N})[0m
Closest candidates are:
  randn(::AbstractRNG, ::Type{T}, [91m::Tuple{Vararg{Int64,N}} where N[39m) where T at random.jl:1292
  randn(::AbstractRNG, ::Type{T}, [91m::Integer[39m, [91m::Integer...[39m) where T at random.jl:1294
  randn(::AbstractRNG) at random.jl:1205
  ...[39m

In [26]:
# load data
include(Knet.dir("data","cifar.jl"))
(xtrn,ytrn,xtst,ytst,lbls)=cifar10()

INFO: Reading cifar-10-binary.tar.gz...


LoadError: [91mSystemError: opening file /home/manuel/.julia/v0.6/Knet/data/cifar/cifar-10-batches-bin/data_batch_2.bin: No such file or directory[39m

In [20]:
# prepare for training
model = optim = nothing; knetgc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum;);

In [22]:
# cold start
for (x,y) in minibatch(xtrn,ytrn,100;shuffle=true,xtype=KnetArray)
    grads = lossgradient(model, x, y)
    update!(model, grads, optim)
end


LoadError: [91mcudnn.cudnnConvolutionForward error 3[39m